In [120]:
import os
import re
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

In [121]:
path = os.path.join(os.getcwd(),"Original Data Sets")
df_list = []

for file in os.listdir(path):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(path,file))
        df["Date"] = file
        df_list.append(df)

df = pd.concat(df_list,axis=0)

In [123]:
df = df[df["nama_kelurahan"] != "TOTAL"]
df = df.reset_index(drop=True)
df.head()

,ID_KEL,Nama_provinsi,nama_kota,nama_kecamatan,nama_kelurahan,SUSPEK,Perawatan RS,Isolasi di Rumah,Suspek Meninggal,Selesai Isolasi,...,Isolasi di Rumah.4,Meninggal,Selesai Isolasi.4,POSITIF,Dirawat,Sembuh,Meninggal.1,Self Isolation,Keterangan,Date
0,3172051003,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,ANCOL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2945,7,2875,30,33,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
1,3173041007,DKI JAKARTA,JAKARTA BARAT,TAMBORA,ANGKE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2870,2,2790,35,43,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
2,3175041005,DKI JAKARTA,JAKARTA TIMUR,KRAMAT JATI,BALE KAMBANG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3196,1,3131,36,28,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
3,3175031003,DKI JAKARTA,JAKARTA TIMUR,JATINEGARA,BALI MESTER,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1361,2,1315,20,24,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...
4,3175101006,DKI JAKARTA,JAKARTA TIMUR,CIPAYUNG,BAMBU APUS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4266,6,4167,36,57,NaN,Standar Kelurahan Data Corona (01 Agustus 2022...


In [124]:
df["Date"] = df["Date"].str.replace("Standar Kelurahan Data Corona ","")
df["Date"] = df["Date"].str.replace(".xlsx","",regex=True)
df["Date"] = df["Date"].str.replace("(","",regex=True)
df["Date"] = df["Date"].str.replace(")","",regex=True)
df["Date"] = df["Date"].str.replace("pukul","")
df["Date"] = df["Date"].str.replace("Pukul","")
df["Date"] = df["Date"].str.replace(".",":",regex=True)
df["Date"] = df["Date"].str.replace("  "," ")

In [125]:
month_mapper = {
    "Januari":"January", 
    "Februari":"February", 
    "Maret":"March", 
    "April":"April",
    "Mei":"May", 
    "Juni":"June", 
    "Juli":"July", 
    "Agustus":"August",
    "September":"September", 
    "Oktober":"October", 
    "November":"November", 
    "Desember":"December"
}
df["Date"] = df["Date"].apply(lambda x: " ".join([month_mapper.get(i,i) for i in x.split()]))
df["Date"] = pd.to_datetime(df["Date"])

In [126]:
df = df[df["Date"].dt.year == 2022]
df = df[df["nama_kecamatan"] != "LUAR DKI JAKARTA"]
df = df[df["nama_kecamatan"] != "PROSES UPDATE DATA"]
df["nama_kecamatan"] = df["nama_kecamatan"].str.replace("KEP.","KEPULAUAN",regex=True)
df = df.sort_values("Date").reset_index(drop=True)
df.head()

,ID_KEL,Nama_provinsi,nama_kota,nama_kecamatan,nama_kelurahan,SUSPEK,Perawatan RS,Isolasi di Rumah,Suspek Meninggal,Selesai Isolasi,...,Isolasi di Rumah.4,Meninggal,Selesai Isolasi.4,POSITIF,Dirawat,Sembuh,Meninggal.1,Self Isolation,Keterangan,Date
0,3172051003,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,ANCOL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1947,0,1919,27,1,NaN,2022-01-01 10:00:00
1,3174041001,DKI JAKARTA,JAKARTA SELATAN,PASAR MINGGU,PASAR MINGGU,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2933,0,2882,51,0,NaN,2022-01-01 10:00:00
2,3171041003,DKI JAKARTA,JAKARTA PUSAT,SENEN,PASEBAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2564,0,2527,37,0,NaN,2022-01-01 10:00:00
3,3173061005,DKI JAKARTA,JAKARTA BARAT,KALI DERES,PEGADUNGAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4466,2,4403,59,2,NaN,2022-01-01 10:00:00
4,3171061002,DKI JAKARTA,JAKARTA PUSAT,MENTENG,PEGANGSAAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1976,0,1927,49,0,NaN,2022-01-01 10:00:00


In [127]:
geolocator = Nominatim(user_agent="MyApp")
latitude = {}
longitude = {}

for kecamatan in df["nama_kecamatan"].unique():
    try:
        location = geolocator.geocode(kecamatan,timeout=None)
        latitude[kecamatan] = location.latitude
        longitude[kecamatan] = location.longitude
    except:
        print(f"Error: Unable to find location for {kecamatan}")

In [128]:
df["latitude_kecamatan"] = df["nama_kecamatan"].apply(lambda x: latitude.get(x))
df["longitude_kecamatan"] = df["nama_kecamatan"].apply(lambda x: longitude.get(x))

In [129]:
df = df[[
    "Nama_provinsi","nama_kota","nama_kecamatan","nama_kelurahan","POSITIF","Dirawat",
    "Sembuh","Meninggal.1","Self Isolation","Date","latitude_kecamatan","longitude_kecamatan"
]]
df = df.rename(columns={"Meninggal.1":"Meninggal"})
df.columns = (df.columns.str.lower()).str.replace(" ","_")
df.head()

,nama_provinsi,nama_kota,nama_kecamatan,nama_kelurahan,positif,dirawat,sembuh,meninggal,self_isolation,date,latitude_kecamatan,longitude_kecamatan
0,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,ANCOL,1947,0,1919,27,1,2022-01-01 10:00:00,-6.129052,106.828972
1,DKI JAKARTA,JAKARTA SELATAN,PASAR MINGGU,PASAR MINGGU,2933,0,2882,51,0,2022-01-01 10:00:00,-6.282674,106.844941
2,DKI JAKARTA,JAKARTA PUSAT,SENEN,PASEBAN,2564,0,2527,37,0,2022-01-01 10:00:00,-6.184971,106.843235
3,DKI JAKARTA,JAKARTA BARAT,KALI DERES,PEGADUNGAN,4466,2,4403,59,2,2022-01-01 10:00:00,-7.235376,110.080258
4,DKI JAKARTA,JAKARTA PUSAT,MENTENG,PEGANGSAAN,1976,0,1927,49,0,2022-01-01 10:00:00,-6.195026,106.832224


In [132]:
df.to_pickle("combined_data.pkl")